# Crawling Kiwilimon

In this file, we'll be downloading and formatting many reciepes from <a href="https://www.kiwilimon.com/" target="_blank">kiwilimon.com<a/>. First try, let's pray 🙏

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

cannot find .env file


In [2]:
# All we need to import

import os
import time
from urllib.parse import urljoin, urlparse

from playwright.async_api import async_playwright

In [3]:
host = 'https://www.kiwilimon.com'
base_dir = 'data/raw'
delay_seconds = 5
bs_parser = 'html.parser'

if not os.path.exists(base_dir):
    os.makedirs(base_dir)

In [4]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()

# Get all categories links

In [5]:
# Xpath
post_xpath = "/html/body/div[10]/div[3]/div[3]/div[1]/div/a" # /html/body/div[10]/div[3]/div[3]/div[1]/div[2]/a


In [6]:
post_links = [] # post of each reciepe
page_url = page.url

while True:
    page_url = f'{host}/recetas'
    print(page_url)
    await page.goto(page_url)
    await page.wait_for_load_state()
    time.sleep(delay_seconds)
    if page_url != page_url:
        break
    for elm in await page.locator("xpath="+post_xpath).element_handles():
        post_url = urljoin(page_url, await elm.get_attribute("href"))
        post_links.append(post_url)
    break

print(len(post_links), len(set(post_links)) )

https://www.kiwilimon.com/recetas
20 20


In [7]:
post_links

['https://www.kiwilimon.com/preferencia/faciles',
 'https://www.kiwilimon.com/recetas/postres',
 'https://www.kiwilimon.com/recetas/carnes-y-aves',
 'https://www.kiwilimon.com/temporada/recetas-a-la-parrilla',
 'https://www.kiwilimon.com/recetas/ensaladas',
 'https://www.kiwilimon.com/recetas/guarniciones',
 'https://www.kiwilimon.com/recetas/pescados-y-mariscos',
 'https://www.kiwilimon.com/recetas/botanas',
 'https://www.kiwilimon.com/recetas/pastas',
 'https://www.kiwilimon.com/recetas/comida-para-ninos',
 'https://www.kiwilimon.com/recetas/postres-para-ninos',
 'https://www.kiwilimon.com/recetas/sopas',
 'https://www.kiwilimon.com/recetas/saludables',
 'https://www.kiwilimon.com/recetas/desayunos',
 'https://www.kiwilimon.com/recetas/panes',
 'https://www.kiwilimon.com/recetas/guarniciones-de-exito',
 'https://www.kiwilimon.com/recetas/salsas',
 'https://www.kiwilimon.com/recetas/bebidas',
 'https://www.kiwilimon.com/recetas/platos-fuertes',
 'https://www.kiwilimon.com/temporada/na

In [8]:
post_links = [{"cat": ' '.join(url[url.rfind("/") + 1:].split('-')).title(), "slug": url[url.rfind("/") + 1:], "link": url, "children": []} for url in post_links]
post_links[:5]

[{'cat': 'Faciles',
  'slug': 'faciles',
  'link': 'https://www.kiwilimon.com/preferencia/faciles',
  'children': []},
 {'cat': 'Postres',
  'slug': 'postres',
  'link': 'https://www.kiwilimon.com/recetas/postres',
  'children': []},
 {'cat': 'Carnes Y Aves',
  'slug': 'carnes-y-aves',
  'link': 'https://www.kiwilimon.com/recetas/carnes-y-aves',
  'children': []},
 {'cat': 'Recetas A La Parrilla',
  'slug': 'recetas-a-la-parrilla',
  'link': 'https://www.kiwilimon.com/temporada/recetas-a-la-parrilla',
  'children': []},
 {'cat': 'Ensaladas',
  'slug': 'ensaladas',
  'link': 'https://www.kiwilimon.com/recetas/ensaladas',
  'children': []}]

## Get recipes link from each categorie

In [9]:
post_xpath = "/html/body/div[10]/div[7]/div[2]/div/a | /html/body/div[10]/div[7]/div[2]/div/div/a"

In [10]:
# Get many links from each categorie

for idx, post in enumerate(post_links):
    page_url = post['link']
    print(page_url)
    await page.goto(page_url)
    await page.wait_for_load_state()
    time.sleep(delay_seconds)
    if page_url != page_url:
        break
        
    # load more recipes clicking the "ver más" button
    for _ in range(4):
        ver_mas_button = page.locator('div:text("Ver más")')
        
        # validate if the button is visible
        is_visible = await ver_mas_button.is_visible()
        if not (await ver_mas_button.count() and is_visible): # avoiding error when there's no more "Ver más" buttons
            break
        await ver_mas_button.click()
        await page.wait_for_load_state()
        time.sleep(1)
    
    # load all the recipe links
    print("loading...")
    for elm in await page.locator("xpath="+post_xpath).element_handles():
        post_url = urljoin(page_url, await elm.get_attribute("href"))
        # print(post_links[idx]['cat'], '-', post_url)
        name = ' '.join(post_url[post_url.rfind("/") + 1:].split('-')).title()
        post_links[idx]['children'].append({'name': name, 'link': post_url})
        
    print(len(post_links[idx]['children']), f"recipes added to '{post_links[idx]['cat']}'.")

print("\nDone!")

https://www.kiwilimon.com/preferencia/faciles
loading...
70 recipes added to 'Faciles'.
https://www.kiwilimon.com/recetas/postres
loading...
70 recipes added to 'Postres'.
https://www.kiwilimon.com/recetas/carnes-y-aves
loading...
70 recipes added to 'Carnes Y Aves'.
https://www.kiwilimon.com/temporada/recetas-a-la-parrilla
loading...
70 recipes added to 'Recetas A La Parrilla'.
https://www.kiwilimon.com/recetas/ensaladas
loading...
70 recipes added to 'Ensaladas'.
https://www.kiwilimon.com/recetas/guarniciones
loading...
70 recipes added to 'Guarniciones'.
https://www.kiwilimon.com/recetas/pescados-y-mariscos
loading...
70 recipes added to 'Pescados Y Mariscos'.
https://www.kiwilimon.com/recetas/botanas
loading...
70 recipes added to 'Botanas'.
https://www.kiwilimon.com/recetas/pastas
loading...
70 recipes added to 'Pastas'.
https://www.kiwilimon.com/recetas/comida-para-ninos
loading...
70 recipes added to 'Comida Para Ninos'.
https://www.kiwilimon.com/recetas/postres-para-ninos
loadi

In [11]:
suma = [len(chil['children']) for chil in post_links]
print("Total recipes:", sum(suma))

Total recipes: 1400


##  Get HTML for each post

In [13]:
for cat in post_links:
    print("Categorie:", cat['cat'])
    print(f"Indexing {len(cat['children'])} pages. Loading...")
    for idx, post in enumerate(cat['children']):
        # print("idx",post['link'])
        # path = get_post_path(post['link'])
        await page.goto(post['link'])
        await page.wait_for_load_state()
        time.sleep(delay_seconds)
        html = await page.content()
        post['html'] = html

print("Done!")

Categorie: Faciles
Indexing 70 pages. Loading...
Categorie: Postres
Indexing 70 pages. Loading...
Categorie: Carnes Y Aves
Indexing 70 pages. Loading...
Categorie: Recetas A La Parrilla
Indexing 70 pages. Loading...
Categorie: Ensaladas
Indexing 70 pages. Loading...
Categorie: Guarniciones
Indexing 70 pages. Loading...
Categorie: Pescados Y Mariscos
Indexing 70 pages. Loading...
Categorie: Botanas
Indexing 70 pages. Loading...
Categorie: Pastas
Indexing 70 pages. Loading...
Categorie: Comida Para Ninos
Indexing 70 pages. Loading...
Categorie: Postres Para Ninos
Indexing 70 pages. Loading...
Categorie: Sopas
Indexing 70 pages. Loading...
Categorie: Saludables
Indexing 70 pages. Loading...
Categorie: Desayunos
Indexing 70 pages. Loading...
Categorie: Panes
Indexing 70 pages. Loading...
Categorie: Guarniciones De Exito
Indexing 70 pages. Loading...
Categorie: Salsas
Indexing 70 pages. Loading...
Categorie: Bebidas
Indexing 70 pages. Loading...
Categorie: Platos Fuertes
Indexing 70 pages. 

In [14]:
await browser.close()
await playwright.stop()

In [19]:
import json

filename = 'recipes.json'
filepath = f"{base_dir}/{filename}"
with open(filepath, "w", encoding="utf-8") as f:
    json.dump(post_links, f, ensure_ascii=False, indent=2)

print(f"Datos guardados en {filepath}")

Datos guardados en data/raw/recipes.json


## Fixing an error
I made a Mistake. I save all the information in one file and that has some problems because the file is too large (2.03 GB). A way to solve it is creating a file for each categorie or creating a file for each page scrapped.

In [1]:
for cat in post_links:
    filename = f"{cat['slug']}.json"
    filepath = f"{base_dir}/{filename}"
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(cat, f, ensure_ascii=False, indent=2)
    
    print(f"Datos guardados en {filepath}")

NameError: name 'post_links' is not defined